Generating trials dataset using resampling procedure.

* Number of resampling iterations: 100 (based on bootstrapping stability analysis)
* Number of trials per resampling: N = 40 (based on data in monkey dataset so the number of trials is from experiments)

In [1]:
import csv
import pickle
import numpy as np

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
#os.chdir(os.path.abspath("C:\\Users\\ipoch\\Documents\\repos\\isttc\\scripts"))
from cfg_global import project_folder_path
from spike_train_utils import get_trials, bin_trials
os.chdir(current_wd)

In [2]:
dataset_folder = project_folder_path + 'results\\synthetic_data\\dataset\\'

### Get data

In [3]:
spike_trains_10min = np.load(dataset_folder + 'spike_trains_tau100ms_alpha0_3_fr3_5hz_len600sec_1000.npy', allow_pickle=True)
print(f'n spike trains {len(spike_trains_10min)}, len {spike_trains_10min[0][-1]/1000}')

n spike trains 1000, len 599.4657698736828


### Make trials

In [4]:
n_trials = 40  
m_iterations = 100

fs = 1000
bin_size = 50 # in ms
signal_len = int(10 * 60 * fs)
trial_len = int(1000 * (fs / 1000)) # 1000ms trials 

In [5]:
trial_dict = {}
trial_binned_dict = {}

for i in range(len(spike_trains_10min)):
    if (i % 100) == 0:
        print(f'Processing unit {i}')
    unit_trial_dict = {}
    unit_trial_binned_dict = {}
    for m in range(m_iterations):
        spikes = np.asarray([int(spike) for spike in spike_trains_10min[i]])
        spikes_trials = get_trials(spikes, signal_len, n_trials, trial_len, verbose_=False)
        spikes_trials_binned = bin_trials(spikes_trials, trial_len, int(bin_size*(fs/1000)))
        unit_trial_dict[m] = spikes_trials
        unit_trial_binned_dict[m] = spikes_trials_binned

    trial_dict[i] = unit_trial_dict
    trial_binned_dict[i] = unit_trial_binned_dict

Processing unit 0
Processing unit 100
Processing unit 200
Processing unit 300
Processing unit 400
Processing unit 500
Processing unit 600
Processing unit 700
Processing unit 800
Processing unit 900


### Save trials 

In [6]:
with open(dataset_folder + 'trial_dict.pkl', 'wb') as f:
    pickle.dump(trial_dict, f)

with open(dataset_folder + 'trial_binned_dict.pkl', 'wb') as f:
    pickle.dump(trial_binned_dict, f)